

What's differentiate metric learning from traditional classification is that we train the model to learn a compact representation

how to have examples of the same classes to be close together and the one f


References
- Original paper:
FaceNet: A Unified Embedding for Face Recognition and Clustering
https://arxiv.org/abs/1503.03832 

- Mining strategies:
https://openaccess.thecvf.com/content_WACV_2020/papers/Xuan_Improved_Embeddings_with_Easy_Positive_Triplet_Mining_WACV_2020_paper.pdf 


In [1]:
 %load_ext autoreload
 %autoreload 2

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [3]:
from tensorflow_similarity.utils import tf_cap_memory
from tensorflow_similarity.losses import TripletLoss
from tensorflow_similarity.layers import MetricEmbedding

In [4]:
import tensorflow_addons as tfa


In [5]:
tf_cap_memory()
print(tf.__version__)

2.3.1


# preparing data

Note: Tensorflow similarity expect y_train to be the examples class as integer so there is no need for processing

In [6]:
# FIXME sampler here and select only half of the class

In [7]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train = tf.constant(x_train / 255.0)
x_test = tf.constant(x_test / 255.0)
x_train.shape

TensorShape([60000, 28, 28])

# model creation

In [15]:
def get_model():
    tf.keras.backend.clear_session()
    inputs = tf.keras.layers.Input(shape=(28,28))
    x = layers.Reshape((28,28,1))(inputs)
    x = layers.Conv2D(16, 3, activation='relu')(x)
    x = layers.Conv2D(32, 3, activation='relu')(x)
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    x = layers.Flatten()(x)
    x = layers.Dense(128, activation='relu')(x)
    outputs = MetricEmbedding(16)(x)
    return Model(inputs, outputs)
model = get_model()
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28)]          0         
_________________________________________________________________
reshape (Reshape)            (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d (Conv2D)              (None, 26, 26, 16)        160       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 32)        4640      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 4608)              0         
_________________________________________________________________
dense (Dense)                (None, 128)              

In [25]:
triplet_loss = TripletLoss(distance=distance,
    positive_mining_strategy=positive_mining_strategy,
    negative_mining_strategy=negative_mining_strategy, reducer='mean')


triplet_loss(y_train[:16], x_train[:16])

(16, 1, 28)
(16, 28, 28)


InvalidArgumentError: in user code:

    C:\Users\elie\AppData\Roaming\Python\Python36\site-packages\tensorflow_addons\utils\keras_utils.py:61 call  *
        return self.fn(y_true, y_pred, **self._fn_kwargs)
    d:\code\similarity\tensorflow_similarity\losses.py:94 triplet_loss  *
        positive_distances = _masked_maximum(pairwise_distances, positive_mask)
    d:\code\similarity\tensorflow_similarity\losses.py:34 _masked_maximum  *
        masked_max = tf.math.multiply(distances - axis_min, mask)
    C:\Users\elie\AppData\Local\Programs\Python\Python36\lib\site-packages\tensorflow\python\util\dispatch.py:201 wrapper  **
        return target(*args, **kwargs)
    C:\Users\elie\AppData\Local\Programs\Python\Python36\lib\site-packages\tensorflow\python\ops\math_ops.py:508 multiply
        return gen_math_ops.mul(x, y, name)
    C:\Users\elie\AppData\Local\Programs\Python\Python36\lib\site-packages\tensorflow\python\ops\gen_math_ops.py:6166 mul
        _ops.raise_from_not_ok_status(e, 

# Compilation

In [18]:
distance = 'cosine' #@param ["cosine"]{allow-input: false}
positive_mining_strategy = 'hard' #@param ["easy", "hard"]{allow-input: false}
negative_mining_strategy = 'hard' #@param ["easy", "hard"]{allow-input: false}
triplet_loss = TripletLoss(distance=distance,
    positive_mining_strategy=positive_mining_strategy,
    negative_mining_strategy=negative_mining_strategy, reducer='mean')

model = get_model()
optimizer = Adam(0.01)
tfa_loss = tfa.losses.TripletSemiHardLoss(distance_metric='L1')
model.compile(loss=triplet_loss, optimizer=optimizer)


EPOCHS = 3 #@param{Integer}
BATCH_SIZE = 128 #@param{Integer}
history = model.fit(x_train, y_train, validation_data=(x_test, y_test),batch_size=BATCH_SIZE, epochs=EPOCHS)

Epoch 1/3
469/469 [==============================] - 2s 4ms/step - loss: 0.6588 - val_loss: 0.6341
Epoch 2/3
469/469 [==============================] - 2s 4ms/step - loss: 0.6330 - val_loss: 0.6340
Epoch 3/3
469/469 [==============================] - 2s 4ms/step - loss: 0.6326 - val_loss: 0.6342


# Training

In [ ]:
a = tf.constant([[0.5, 0.2, 0.1], [0.4, 0.3, 0,2]])

tfa.losses.metric_learning.pairwise_distance(a)